In [1]:
from langchain_core.messages import HumanMessage, ToolMessage, SystemMessage
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import add_messages, StateGraph, END # add_message: to update graph state
from langgraph.checkpoint.memory import MemorySaver
from typing import TypedDict, Annotated, List, Optional, Dict # to define state of graph
from langchain_openai import ChatOpenAI

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
# Custom Tools

from typing import Optional
import pandas as pd
import datetime

from langchain_core.callbacks import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from langchain_core.tools import BaseTool
from langchain_core.tools.base import ArgsSchema
from pydantic import BaseModel, Field

class DataInput(BaseModel):
    a: str = Field(description="port")
    b: datetime.date = Field(description="end_date")


class DataFetchTool(BaseTool):
    name: str = "data_fetch_tool"
    description: str = "useful for when you need are asked to fetch dataframe/data on a portfolio"
    args_schema: Optional[ArgsSchema] = DataInput
    return_direct: bool = True

    def _run(
        self, a: int, b: int, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> pd.DataFrame:
        """Use the tool."""
        return pd.DataFrame({
            "a": [1,2,3,4,5,6],
            "b": [b + datetime.timedelta(days=i) for i in range(6)]
        })

    async def _arun(
        self,
        a: str,
        b: datetime.date,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> int:
        """Use the tool asynchronously."""
        return self._run(a, b, run_manager=run_manager.get_sync())

In [4]:
model = ChatOpenAI(model="gpt-4.1-nano")
search_tool = TavilySearchResults(max_results=1)
db_tool = DataFetchTool()
tools = [search_tool, db_tool]
memory = MemorySaver()
llm_with_tools = model.bind_tools(tools=tools)

#### Graph

In [ ]:
# Graph State
def add_dataframes(current_dfs: Optional[Dict[str, pd.DataFrame]], new_dfs: Dict[str, pd.DataFrame]):
    if current_dfs is None:
        return new_dfs
    return {**current_dfs, **new_dfs}

dataframes_dict = {}

class State(TypedDict):
    messages: Annotated[list, add_messages]

In [6]:
# Defining Nodes

async def model(state: State):
    messages = state["messages"]
    if isinstance(messages[-1], SystemMessage):
        return state
    result = await llm_with_tools.ainvoke(state["messages"])
    return {
        "messages": [result], 
    }

async def tool_node(state):
    """Custom tool node that handles tool calls from the LLM."""
    
    tool_calls = state["messages"][-1].tool_calls # Get the tool calls from the last message
    tool_messages = [] # Initialize list to store tool messages
    
    for tool_call in tool_calls:
        tool_name = tool_call["name"]
        tool_args = tool_call["args"]
        tool_id = tool_call["id"]
        
        # Handle the search tool
        if tool_name == "tavily_search_results_json":
            search_results = await search_tool.ainvoke(tool_args)
            
            tool_message = ToolMessage(
                content=str(search_results),
                tool_call_id=tool_id,
                name=tool_name
            )
            
            tool_messages.append(tool_message)
            
        if tool_name == "data_fetch_tool":
            df = await db_tool.ainvoke(tool_args)
            
            tool_message = ToolMessage(
                content=df,
                tool_call_id=tool_id,
                name=tool_name
            )
            tool_messages.append(tool_message)
            dataframes_dict[f"{tool_name}_{tool_args}"] = df

    return {"messages": tool_messages}

In [7]:
# Router

async def tools_router(state: State):
    last_message = state["messages"][-1]

    if(hasattr(last_message, "tool_calls") and len(last_message.tool_calls) > 0):
        return "tool_node"
    return END

In [8]:
# Building graph

graph_builder = StateGraph(State)

# adding nodes
graph_builder.add_node("model", model)
graph_builder.add_node("tool_node", tool_node)

graph_builder.set_entry_point("model")

# adding edges
graph_builder.add_conditional_edges("model", tools_router)
graph_builder.add_edge("tool_node", "model")

graph = graph_builder.compile(checkpointer=memory)

In [9]:
config = {
    "configurable": {
        "thread_id": 1
    }
}

# Adding a SystemMessage to indicate the model is a finance bot
initial_state = {
    "messages": [SystemMessage(content="You are a finance bot.")]
}

# Invoke the graph with the initial state
response = await graph.ainvoke(initial_state, config=config)
response

response = await graph.ainvoke({
    "messages": [HumanMessage(content="Do you know my name")], 
}, config=config)

response

{'messages': [SystemMessage(content='You are a finance bot.', additional_kwargs={}, response_metadata={}, id='3c0a7a8b-6d78-4f08-a2ff-3877095fa600'),
  HumanMessage(content='Do you know my name', additional_kwargs={}, response_metadata={}, id='15ac22e4-4152-40c9-8313-c41852143987'),
  AIMessage(content="I don't know your name unless you tell me. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 136, 'total_tokens': 155, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_8fd43718b3', 'id': 'chatcmpl-BWuxs31N6bENuSxpNnVFBELGbAK8E', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--a2ff9ceb-004a-4f99-84f4-f75be847e0e0-0', usage_metadata={'input_toke

In [10]:
response = await graph.ainvoke({
    "messages": [HumanMessage(content="can you give me data for port ABC-AGG, and end date 20 march 2025")], 
}, config=config)

response

{'messages': [SystemMessage(content='You are a finance bot.', additional_kwargs={}, response_metadata={}, id='3c0a7a8b-6d78-4f08-a2ff-3877095fa600'),
  HumanMessage(content='Do you know my name', additional_kwargs={}, response_metadata={}, id='15ac22e4-4152-40c9-8313-c41852143987'),
  AIMessage(content="I don't know your name unless you tell me. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 136, 'total_tokens': 155, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_8fd43718b3', 'id': 'chatcmpl-BWuxs31N6bENuSxpNnVFBELGbAK8E', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--a2ff9ceb-004a-4f99-84f4-f75be847e0e0-0', usage_metadata={'input_toke

In [15]:
response = await graph.ainvoke({
    "messages": [HumanMessage(content="can you display the df a had asked you")], 
}, config=config)

In [16]:
response

{'messages': [SystemMessage(content='You are a finance bot.', additional_kwargs={}, response_metadata={}, id='3c0a7a8b-6d78-4f08-a2ff-3877095fa600'),
  HumanMessage(content='Do you know my name', additional_kwargs={}, response_metadata={}, id='15ac22e4-4152-40c9-8313-c41852143987'),
  AIMessage(content="I don't know your name unless you tell me. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 136, 'total_tokens': 155, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_8fd43718b3', 'id': 'chatcmpl-BWuxs31N6bENuSxpNnVFBELGbAK8E', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--a2ff9ceb-004a-4f99-84f4-f75be847e0e0-0', usage_metadata={'input_toke

In [46]:
config = {
    "configurable": {
        "thread_id": 2
    }
}

# Use async for to iterate over the async generator
async for event in graph.astream_events({
    "messages": [HumanMessage(content="what's the current situation between india and pakistan")],
}, config=config, version="v2"):
    print(event)

{'event': 'on_chain_start', 'data': {'input': {'messages': [HumanMessage(content="what's the current situation between india and pakistan", additional_kwargs={}, response_metadata={})]}}, 'name': 'LangGraph', 'tags': [], 'run_id': '14ab3045-2452-4f95-891f-49e7e583e444', 'metadata': {'thread_id': 2}, 'parent_ids': []}
{'event': 'on_chain_start', 'data': {'input': {'messages': [HumanMessage(content="what's the current situation between india and pakistan", additional_kwargs={}, response_metadata={}, id='51ded05c-0f4c-4c7b-97e9-760b777d6410')]}}, 'name': 'model', 'tags': ['graph:step:1'], 'run_id': '1642b11e-5b5e-4352-a82d-11a11412c5cb', 'metadata': {'thread_id': 2, 'langgraph_step': 1, 'langgraph_node': 'model', 'langgraph_triggers': ('branch:to:model',), 'langgraph_path': ('__pregel_pull', 'model'), 'langgraph_checkpoint_ns': 'model:ac805cab-97b6-e69d-f73d-f1b6fbfcd022'}, 'parent_ids': ['14ab3045-2452-4f95-891f-49e7e583e444']}
{'event': 'on_chat_model_start', 'data': {'input': {'message